# Prepare for colab

In [1]:
!pip install xgboost -U
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 49 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Upload API Token

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/backups/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


## Download and unzip dataset

In [3]:
!kaggle datasets download -d ryuina/amex-fe-plus
!unzip /content/amex-fe-plus.zip

 99% 3.83G/3.85G [00:16<00:00, 358MB/s]
100% 3.85G/3.85G [00:16<00:00, 247MB/s]
Archive:  /content/amex-fe-plus.zip
  inflating: test_fe_plus_plus.parquet  
  inflating: train_fe_plus_plus.parquet  


In [4]:
!kaggle competitions download -c amex-default-prediction -f sample_submission.csv
!unzip /content/sample_submission.csv.zip

 28% 9.00M/32.4M [00:00<00:00, 34.3MB/s]
100% 32.4M/32.4M [00:00<00:00, 97.3MB/s]
Archive:  /content/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [5]:
!kaggle competitions download -c amex-default-prediction -f train_labels.csv
!unzip /content/train_labels.csv.zip

 56% 9.00M/16.2M [00:00<00:00, 40.3MB/s]
100% 16.2M/16.2M [00:00<00:00, 65.3MB/s]
Archive:  /content/train_labels.csv.zip
  inflating: train_labels.csv        


# Training

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot
import os
import gc; gc.enable()

from sklearn.preprocessing import LabelEncoder

In [2]:
VER=1
SEED = 42
FOLDS = 5

cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(SEED)

In [3]:
train = pd.read_parquet("/content/train_fe_plus_plus.parquet")
# train = train.reset_index(level=0)

In [4]:
# https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7963
# Round last float features to 2 decimal place
float_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
float_cols = [col for col in float_cols if 'last' in col]

train[float_cols] = train[float_cols].round(2)
train.fillna(-127, inplace=True)

In [5]:
train = train.set_index('customer_ID')
train = train.sort_index().reset_index()
FEATURES = train.drop(columns=['customer_ID', 'target']).columns

In [6]:
# LOAD XGB LIBRARY
from sklearn.model_selection import KFold
import xgboost as xgb
print('XGB Version',xgb.__version__)

# XGB MODEL PARAMETERS
xgb_parms = {
    'max_depth':7,
    'eta':0.02,
    'subsample':0.88,
    'colsample_bytree':0.5,
    'min_child_weight':8,
#     'booster': 'dart',
#     'max_delta_step': 16,
    'gamma':1.5,
    'lambda':70,
#     'reg_alpha':2,
#     'reg_lambda':2,
#     'scale_pos_weight':2.5,
#     'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

XGB Version 1.6.1


In [7]:
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
import numpy as np

def amex_metric_numpy(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

def xgb_amex(y_pred, y_true):
    return 'amex', amex_metric_numpy(y_true.get_label(), y_pred)

In [8]:
importances = []
oof = []
gc.collect()

skf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train.target )):
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print('#'*25)
    
    # TRAIN, VALID, TEST FOR FOLD K
    y_valid = train.loc[valid_idx, 'target']
    
    dtrain = xgb.DMatrix(data=train.loc[train_idx, FEATURES], label=train.loc[train_idx, 'target'])
    dvalid = xgb.DMatrix(data=train.loc[valid_idx, FEATURES], label=y_valid)

    # TRAIN MODEL FOLD K
    model = xgb.train(xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'eval')],
                num_boost_round=15000,
                early_stopping_rounds=500,
                maximize=True,
                custom_metric= xgb_amex,
                verbose_eval=200) 
    model.save_model(f'XGB_v{VER}_fold{fold}.xgb')
    
    # GET FEATURE IMPORTANCE FOR FOLD K
    dd = model.get_score(importance_type='weight')
    df = pd.DataFrame({'feature':dd.keys(),f'importance_{fold}':dd.values()})
    importances.append(df)
            
    # INFER OOF FOLD K
    oof_preds = model.predict(dvalid)
    acc = amex_metric_numpy(y_valid.values, oof_preds)
    print('Kaggle Metric =',acc,'\n')
    
    # SAVE OOF
    df = train.loc[valid_idx, ['customer_ID','target'] ].copy()
    df['oof_pred'] = oof_preds
    oof.append( df )

    del dtrain, dd, df
    del y_valid, dvalid, model
    _ = gc.collect()
    
print('#'*25)
oof = pd.concat(oof,axis=0,ignore_index=True).set_index('customer_ID')
acc = amex_metric_numpy(oof.target.values, oof.oof_pred.values)
print('OVERALL CV Kaggle Metric =',acc)

#########################
### Fold 1
### Train size 367130 Valid size 91783
#########################
[0]	train-logloss:0.68036	train-amex:0.69638	eval-logloss:0.68040	eval-amex:0.69152
[200]	train-logloss:0.22885	train-amex:0.78658	eval-logloss:0.23620	eval-amex:0.77435
[400]	train-logloss:0.21053	train-amex:0.80711	eval-logloss:0.22389	eval-amex:0.78393
[600]	train-logloss:0.20211	train-amex:0.82005	eval-logloss:0.22061	eval-amex:0.78815
[800]	train-logloss:0.19557	train-amex:0.83136	eval-logloss:0.21892	eval-amex:0.79011
[1000]	train-logloss:0.19012	train-amex:0.84068	eval-logloss:0.21807	eval-amex:0.79269
[1200]	train-logloss:0.18518	train-amex:0.84908	eval-logloss:0.21752	eval-amex:0.79331
[1400]	train-logloss:0.18049	train-amex:0.85772	eval-logloss:0.21709	eval-amex:0.79351
[1600]	train-logloss:0.17608	train-amex:0.86565	eval-logloss:0.21683	eval-amex:0.79427
[1800]	train-logloss:0.17184	train-amex:0.87318	eval-logloss:0.21668	eval-amex:0.79426
[2000]	train-logloss:0.16787	train-

In [9]:
# CLEAN RAM
del train
_ = gc.collect()

In [10]:
import matplotlib.pyplot as plt

df = importances[0].copy()
for k in range(1,FOLDS): df = df.merge(importances[k], on='feature', how='left')
df['importance'] = df.iloc[:,1:].mean(axis=1)
df = df.sort_values('importance',ascending=False)
df.to_csv(f'xgb_feature_importance_v{VER}.csv',index=False)

In [11]:
NUM_FEATURES = 20
plt.figure(figsize=(10,5*NUM_FEATURES//10))
plt.barh(np.arange(NUM_FEATURES,0,-1), df.importance.values[:NUM_FEATURES])
plt.yticks(np.arange(NUM_FEATURES,0,-1), df.feature.values[:NUM_FEATURES])
plt.title(f'XGB Feature Importance - Top {NUM_FEATURES}')
plt.show()

# Test part

In [12]:
# CALCULATE SIZE OF EACH SEPARATE TEST PART
def get_rows(customers, test, NUM_PARTS = 4, verbose = ''):
    chunk = len(customers)//NUM_PARTS
    if verbose != '':
        print(f'We will process {verbose} data as {NUM_PARTS} separate parts.')
        print(f'There will be {chunk} customers in each part (except the last part).')
        print('Below are number of rows in each part:')
    rows = []

    for k in range(NUM_PARTS):
        if k==NUM_PARTS-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = test.loc[test.customer_ID.isin(cc)].shape[0]
        rows.append(s)
    if verbose != '': print( rows )
    return rows,chunk

In [13]:
# COMPUTE SIZE OF 4 PARTS FOR TEST DATA
NUM_PARTS = 4

test = pd.read_parquet("/content/test_fe_plus_plus.parquet")
test[float_cols] = test[float_cols].round(2)
test.fillna(-127, inplace=True)

customers = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
rows,num_cust = get_rows(customers, test[['customer_ID']], NUM_PARTS = NUM_PARTS, verbose = 'test')

We will process test data as 4 separate parts.
There will be 231155 customers in each part (except the last part).
Below are number of rows in each part:
[231155, 231155, 231155, 231156]


In [14]:
# INFER TEST DATA IN PARTS
skip_rows = 0
skip_cust = 0
test_preds = []

for k in range(NUM_PARTS):
    # READ PART OF TEST DATA
    print(f'\nReading test data...')
    test_copy = test.iloc[skip_rows:skip_rows+rows[k]].copy()
    test_copy = test_copy.set_index('customer_ID')
    skip_rows += rows[k]
    print(f'=> Test part {k+1} has shape', test_copy.shape )
    
    # PROCESS AND FEATURE ENGINEER PART OF TEST DATA
    if k==NUM_PARTS-1: test_copy = test_copy.loc[customers[skip_cust:]]
    else: test_copy = test_copy.loc[customers[skip_cust:skip_cust+num_cust]]
    skip_cust += num_cust
    
    # TEST DATA FOR XGB
    X_test = test_copy[FEATURES]
    dtest = xgb.DMatrix(data=X_test)
    test_copy = test_copy[['P_2_mean']] # reduce memory
    del X_test
    gc.collect()

    # INFER XGB MODELS ON TEST DATA
    model = xgb.Booster()
    model.load_model(f'XGB_v{VER}_fold0.xgb')
    preds = model.predict(dtest)
    for f in range(1,FOLDS):
        model.load_model(f'XGB_v{VER}_fold{f}.xgb')
        preds += model.predict(dtest)
    preds /= FOLDS
    test_preds.append(preds)

    # CLEAN MEMORY
    del test_copy, dtest, model
    _ = gc.collect()

del test
_ = gc.collect()


Reading test data...
=> Test part 1 has shape (231155, 1460)

Reading test data...
=> Test part 2 has shape (231155, 1460)

Reading test data...
=> Test part 3 has shape (231155, 1460)

Reading test data...
=> Test part 4 has shape (231156, 1460)


In [15]:
# WRITE SUBMISSION FILE
test = pd.DataFrame(index=customers,data={'prediction': np.concatenate(test_preds)})
sub = pd.read_csv('/content/sample_submission.csv')[['customer_ID']]
sub['customer_ID_hash'] = sub['customer_ID'].copy()
sub = sub.set_index('customer_ID_hash')
sub = sub.merge(test[['prediction']], left_index=True, right_index=True, how='left')
sub = sub.reset_index(drop=True)

# DISPLAY PREDICTIONS
sub.to_csv(f'submission_xgb_v{VER}_seed{SEED}_fold{FOLDS}.csv',index=False)
print('Submission file shape is', sub.shape )
sub.head()

Submission file shape is (924621, 2)


,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.030717
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000779
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.061296
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.251855
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.907319


In [17]:
# PLOT PREDICTIONS
plt.hist(sub.prediction, bins=100)
plt.title('Test Predictions')
plt.show()

# Submit to Kaggle from Colab

In [18]:
# Submit
!kaggle competitions submit -c amex-default-prediction -f submission_xgb_v1_seed42_fold5.csv -m "From colab seed 42 fold 5 lag plus"

100% 67.6M/67.6M [00:01<00:00, 65.2MB/s]
Successfully submitted to American Express - Default Prediction

In [19]:
# Run after few seconds
!kaggle competitions submissions amex-default-prediction

fileName                            date                 description                                               status    publicScore  privateScore  
----------------------------------  -------------------  --------------------------------------------------------  --------  -----------  ------------  
submission_xgb_v1_seed42_fold5.csv  2022-07-20 16:58:31  From colab seed 42 fold 5 lag plus                        complete  0.794                      
submission_cat_v1_seed42_fold5.csv  2022-07-19 17:31:52  From colab seed 42 fold 5 lag plus                        complete  0.795                      
Amex Default Prediction - Ensemble  2022-07-19 16:37:40  Notebook Amex Default Prediction - Ensemble | Version 4   complete  0.798                      
Amex Default Prediction - Ensemble  2022-07-19 16:34:17  Notebook Amex Default Prediction - Ensemble | Version 2   complete  0.798                      
submission_cat_v1_seed42_fold5.csv  2022-07-19 12:11:59  From colab seed 42 fold 5